In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf


Clone tensorflow to use the object detection API


In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 41012, done.
remote: Total 41012 (delta 0), reused 0 (delta 0), pack-reused 41012
Receiving objects: 100% (41012/41012), 548.83 MiB | 15.08 MiB/s, done.
Resolving deltas: 100% (27761/27761), done.


Install dependencies needed for object detection

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,791 kB of archives.
After this operation, 7,807 kB of additional disk space will be used.
Get:1 htt

Convert the proto files to python files

In [1]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.


bash: line 1: cd: models/research/: No such file or directory
object_detection/protos/*.proto: No such file or directory
cp: cannot stat 'object_detection/packages/tf2/setup.py': No such file or directory
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


Add object detection program to the python path

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

Setup for object detection 

In [ ]:
!python setup.py build
!python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/model_lib_v2.py -> build/lib/object_detection
copying object_detection/model_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/model_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/model_main_tf2.py -> build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object_detection
copying object_detection/exporter_lib_v2.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying

Install tf_slim dependency

In [ ]:
!pip3 install tf_slim

     |████████████████████████████████| 358kB 2.8MB/s 


Run the model_builder_test to make sure we installed everything correctly

In [ ]:

%cd /content/models/research/object_detection/builders/
!python model_builder_test.py

/content/models/research/object_detection/builders


Get or create your tfrecords for training and testing. 

In [ ]:
%cd /content/
if(os.path.exists('/content/cattle_counter_data')):
  !cd /content/cattle_counter_data && git pull
else:
  !cd /content/ && git clone https://Jacob_Boesch@bitbucket.org/Jacob_Boesch/cattle_counter_data_tf2.git
  !mv cattle_counter_data_tf2 cattle_counter_data

/content
Cloning into 'cattle_counter_data_tf2'...
remote: Counting objects: 12, done.
remote: Compressing objects: 100% (10/10), done.
tcmalloc: large alloc 1932320768 bytes == 0x55fc7ea5c000 @  0x7fbe28ed21e7 0x55fc7cb057b5 0x55fc7cb0586d 0x55fc7ca0627b 0x55fc7ca06900 0x55fc7c986a15 0x55fc7c9876d5 0x55fc7c9866da 0x7fbe28219b97 0x55fc7c98672a
remote: Total 12 (delta 2), reused 0 (delta 0)
Unpacking objects: 100% (12/12), done.
Checking out files: 100% (5/5), done.


In [ ]:

test_record_fname = '/content/cattle_counter_data/cattle.tfrecord'
train_record_fname = '/content/cattle_counter_data/cattle_val.tfrecord'
label_map_pbtxt_fname = '/content/cattle_counter_data/cattle.pbtxt'

In [ ]:
#download base training configuration file
model_name = 'ssd_mobilenet_v3_small_coco_2020_01_14'
download_config = 'http://download.tensorflow.org/models/object_detection/{model_name}.tar.gz'.format(model_name=model_name)
!wget {download_config}
!tar -zxvf { model_name + '.tar.gz'}

--2020-08-07 11:40:17--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.188.128, 2404:6800:4008:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.188.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27341125 (26M) [application/gzip]
Saving to: ‘ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz’

ssd_mobilenet_v3_sm 100%[===================>]  26.07M  65.3MB/s    in 0.4s    

2020-08-07 11:40:18 (65.3 MB/s) - ‘ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz’ saved [27341125/27341125]

ssd_mobilenet_v3_small_coco_2020_01_14/
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.index
ssd_mobilenet_v3_small_coco_2020_01_14/frozen_inference_graph.pb
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.data-00000-of-00001
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.meta
ssd_mobilenet_v3_small_coco_2020_01_14/pipeline.co

In [ ]:
#prepare
pipeline_fname = '/content/' + model_name + '/pipeline.config'
fine_tune_checkpoint = '/content/' + model_name + '/model.ckpt'
label_map_pbtxt_fname = '/content/cattle_counter_data/cattle.pbtxt'
num_classes = 1
batch_size = 64
num_steps = 2000

In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd {'/content/' + model_name + '/'}
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        'input_path: "PATH_TO_BE_CONFIGURED\/.*_train.*"', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        'input_path: "PATH_TO_BE_CONFIGURED\/.*_val.*"', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
     #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    
    s = re.sub(
        'learning_rate_base: \d\.\d*', 'learning_rate_base: {}'.format(0.001), s)
    s = re.sub(
        'warmup_learning_rate: \d\.\d*', 'warmup_learning_rate: {}'.format(0.001), s)
    """
    s = re.sub(
        'width: \d+', 'width: 640', s
    )

    s = re.sub('height: \d+', 'height: 640', s)
  """
    f.write(s)



/content/ssd_mobilenet_v3_small_coco_2020_01_14
writing custom configuration file


In [ ]:
%cat {pipeline_fname}

# SSDLite with Mobilenet v3 large feature extractor.
# Trained on COCO14, initialized from scratch.
# TPU-compatible.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: tru

Train the model

In [ ]:
!cp /content/models/research/object_detection/legacy/train.py /content/train.py
%cd /content/
!python /content/train.py --logtostderr --train_dir=training/ --pipeline_config_path={pipeline_fname}

Streaming output truncated to the last 5000 lines.
I0807 13:44:25.981510 139735103084416 learning.py:512] global step 1922: loss = 0.4080 (0.131 sec/step)
INFO:tensorflow:global step 1922: loss = 0.9616 (0.091 sec/step)
I0807 13:44:26.085717 139735103084416 learning.py:512] global step 1922: loss = 0.9616 (0.091 sec/step)
INFO:tensorflow:global step 1922: loss = 0.7541 (0.149 sec/step)
I0807 13:44:26.235680 139735103084416 learning.py:512] global step 1922: loss = 0.7541 (0.149 sec/step)
INFO:tensorflow:global step 1922: loss = 0.8893 (0.136 sec/step)
I0807 13:44:26.372843 139735103084416 learning.py:512] global step 1922: loss = 0.8893 (0.136 sec/step)
INFO:tensorflow:global step 1922: loss = 0.4971 (0.144 sec/step)
I0807 13:44:26.518339 139735103084416 learning.py:512] global step 1922: loss = 0.4971 (0.144 sec/step)
INFO:tensorflow:global step 1922: loss = 0.6325 (0.110 sec/step)
I0807 13:44:26.629759 139735103084416 learning.py:512] global step 1922: loss = 0.6325 (0.110 sec/step)


##EXporting inference graph

In [ ]:
%cd /content/models/research/object_detection
!python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path {pipeline_fname} --trained_checkpoint_prefix /content/training/model.ckpt-2000 --output_directory /content/trained_model

/content/models/research/object_detection
Instructions for updating:
Please use `layer.__call__` method instead.
W0807 13:51:35.512678 140504388228992 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0807 13:51:37.326531 140504388228992 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0807 13:51:37.405172 140504388228992 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0807 13:51:37.590405 140504388228992 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INF

#Testing the Model





In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



### Model preparation variable
MODEL_NAME = '/content/trained_model'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/cattle_counter_data/cattle.pbtxt'
NUM_CLASSES = 1


### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)



PATH_TO_TEST_IMAGES_DIR = '/content/cattle_counter_data/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'test.jpg') ]
IMAGE_SIZE = (320, 320)




def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
    
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict




for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  print(output_dict)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=1)
  display(Image.fromarray(image_np))
  



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=/content/trained_model/pipeline.config \
--trained_checkpoint_prefix=/content/trained_model/model.ckpt \
--output_directory=/content/tflite \
--add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0807 13:52:05.343803 140272920217472 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0807 13:52:07.114973 140272920217472 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0807 13:52:07.184513 140272920217472 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0807 13:52:07.252123 140272920217472 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [ ]:
!sudo apt install curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!sudo mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update && sudo apt install bazel
!sudo apt install bazel-3.1.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.9).
gnupg is already the newest version (2.2.4-1ubuntu1.2).
gnupg set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:2 https://storage.googleapis.com/bazel-apt stable InRelease [2,256 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute

In [ ]:
%cd /content/
!git clone --recurse-submodules https://github.com/tensorflow/tensorflow.git

/content
Cloning into 'tensorflow'...
remote: Enumerating objects: 965697, done.
remote: Total 965697 (delta 0), reused 0 (delta 0), pack-reused 965697
Receiving objects: 100% (965697/965697), 567.77 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (786147/786147), done.
Checking out files: 100% (22958/22958), done.


In [ ]:
%cd /content/tensorflow/
!bazel run -c opt tensorflow/lite/toco:toco -- \
--input_file=/content/tflite/tflite_graph.pb \
--output_file=/content/tflite/detect.tflite \
--input_shapes=1,320,320,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=FLOAT \
--mean_values=128 \
--std_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

Streaming output truncated to the last 5000 lines.
    Compiling external/aws/aws-cpp-sdk-core/source/http/URI.cpp; 0s local
    @aws//:aws; 0s local
[1,613 / 2,713] 4 actions, 3 running
    Compiling external/aws/aws-cpp-sdk-core/source/http/URI.cpp; 1s local
    @aws//:aws; 0s local
    @aws//:aws; 0s local
[1,614 / 2,713] 4 actions, 3 running
    Compiling external/aws/aws-cpp-sdk-core/source/http/URI.cpp; 1s local
    @aws//:aws; 0s local
    @com_google_protobuf//:protobuf; 0s local
[1,616 / 2,713] 4 actions, 3 running
    @aws//:aws; 0s local
    @com_google_protobuf//:protobuf; 0s local
    Compiling external/boringssl/src/crypto/x509/x509cset.c; 0s local
[1,621 / 2,713] 4 actions running
    @aws//:aws; 0s local
    Compiling external/boringssl/src/crypto/x509v3/v3_crld.c; 0s local
    @aws//:aws; 0s local
[1,624 / 2,713] 4 actions, 3 running
    @aws//:aws; 0s local
    @aws//:aws; 0s local
    Compiling tensorflow/core/lib/random/distribution_sampler.cc; 0s local
[1,625 / 2,7

In [ ]:
interpreter = tf.lite.Interpreter(model_path='/content/tflite/detect.tflite')
interpreter.allocate_tensors()

print(interpreter.get_input_details())

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(interpreter.get_output_details())

from keras.preprocessing import image
import numpy as np
import tensorflow as tf
# predicting images
path = '/content/cattle_counter_data/test.jpg'
img = image.load_img(path, target_size=(320, 320))
x = image.img_to_array(img, dtype=np.float32) 
x = np.expand_dims(x, axis=0)

interpreter.set_tensor(input_index,x)
interpreter.invoke()
prediction = interpreter.get_tensor(output_index)
print(prediction)

[{'name': 'normalized_input_image_tensor', 'index': 381, 'shape': array([  1, 320, 320,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'TFLite_Detection_PostProcess', 'index': 373, 'shape': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}, {'name': 'TFLite_Detection_PostProcess:1', 'index': 374, 'shape': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}, {'name': 'TFLite_Detection_PostProcess:2', 'index': 375, 'shape': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}, {'name': 'TFLite_Detection_PostProcess:3', 'index': 376, 'shape': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


Using TensorFlow backend.


[[[ 0.41983387  0.02924089  0.6728326   0.08463179]
  [ 0.25900534  0.02451942  0.5135829   0.08401314]
  [-0.31423882  0.23418024  1.3491167   0.7630316 ]
  [ 0.3643055   0.03364203  0.61661357  0.08502508]
  [ 0.11597872  0.06497172  0.2524802   0.09482805]
  [ 0.18599942 -0.20001318  0.7939026   1.2907662 ]
  [ 0.39335975  0.03165036  0.5809714   0.06599544]
  [ 0.5191964   0.03102319  0.7116872   0.09784792]
  [ 0.01231836  0.0362398   0.30769843  0.11153974]
  [ 0.00555007  0.01760946  0.9760647   0.9215371 ]]]
